<a href="https://colab.research.google.com/github/quaziharis/E-Commerce/blob/master/Game_Sentiment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
!pip install nltk
!pip install spacy
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', None)
import seaborn as sns
import os
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
game_overview = pd.read_csv('/content/game_overview.csv')

In [0]:
df = pd.merge(df,game_overview[['title','developer', 'publisher', 'tags','overview']],
                 on='title', 
                 how='left')

In [0]:
test = pd.merge(test,game_overview[['title','developer', 'publisher', 'tags','overview']],
                 on='title', 
                 how='left')

In [0]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata
nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)
#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text
#Removing accented characters
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#Expanding Contractions
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

#Removing Special Characters
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [0]:
#Stemming
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

#Removing Stop Words

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


In [0]:
  df.head(2)

,review_id,title,year,user_review,user_suggestion,developer,publisher,tags,overview,full_text,clean_text
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...,Spooky's Jump Scare Mansion. I'm scared and he...,spookys jump scare mansion scared hear creepy ...
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...,"Spooky's Jump Scare Mansion. Best game, more b...",spookys jump scare mansion good game well sam ...


In [0]:
# combining headline and article text
df['full_text'] = df["title"].map(str)+ '. ' + df["user_review"]+ '. ' + df["overview"]+ '. ' + df["tags"].map(str)+ '. ' +df['publisher']+'. '+df['developer']

In [0]:
test['full_text'] = test["title"].map(str)+ '. ' + test["user_review"]+ '. ' + test["overview"]+ '. ' + test["tags"].map(str)+ '. ' +test['publisher']+'. '+test['developer']

In [0]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
    return normalized_corpus

# pre-process text and store the same
df['clean_text'] = normalize_corpus(df['full_text'])
norm_corpus = list(df['clean_text'])

# show a sample news article
df.iloc[1][['full_text', 'clean_text']].to_dict()

{'clean_text': 'spookys jump scare mansion good game well sam pepper youtube account need play computersome extra pants pro scary hell fun adventure spooky forgot mention scary hellyou get pant brief wardrobe time consume bored con buy pant brief not download yet survive room cute terror break cuteness start fade run life unspeakable hideous shake writhe bowel house wait wait hunger meet long finally meet show flexible skin soak blood brave journey set beat impossible insane incorporeal [ horror free play cute first person singleplayer psychological horror indie adventure dark funny atmospheric action walk simulator survival survival horror anime gore comedy multiplayer illuminati ] lag studio lag studio',
 'full_text': "Spooky's Jump Scare Mansion. Best game, more better than Sam Pepper's YouTube account. 10/10What you'll need to play:A computerSome extra pants.Pros:Scary as hell.Fun.Adventure.Spooky.Did I forgot to mention that its scary as hell?You'll get more pants/briefs in your w

In [0]:
test['clean_text'] = normalize_corpus(test['full_text'])
norm_corpus = list(test['clean_text'])

# show a sample news article
test.iloc[1][['full_text', 'clean_text']].to_dict()

{'clean_text': 'counter strike global offensive would not recommend get current state csgo hit rock bottom valve miss promise game change update lack effort combat terrible cheating problem bug game year still not fix cheating control past year playing cheating get bad bad point end play cheater almost every game valve implement new trust matchmaking system combat issue lackluster algorithm still somehow think put year gameplay hour people play brand new account make sense really not know still put effort play game anymore usually end regret guess wish could somehow fun use play game seem day long go counter strike global offensive cs go expand upon team base action gameplay pioneer launch year ago cs go feature new map character weapon game mode deliver update version classic cs content de_dust etc counter strike take gaming industry surprise unlikely mod become played online pc action game world almost immediately release august say doug lombardi valve past year continue one play gam

In [0]:
test_all_Game=test['clean_text']
train_all_Game=df[['clean_text','user_suggestion']]

In [0]:
my_df = pd.read_csv('/content/Videogame_clean_train.csv')
my_df.head()

,clean_text,user_suggestion
0,spookys jump scare mansion scared hear creepy ...,1
1,spookys jump scare mansion good game well sam ...,1
2,spookys jump scare mansion littly iffy control...,1
3,spookys jump scare mansion great game fun colo...,1
4,spookys jump scare mansion not many game cute ...,1


In [0]:
test = pd.read_csv('/content/Videogame_clean_test.csv')

In [0]:
my_df = pd.read_csv('/content/Videogame_clean_train.csv')
test = pd.read_csv('/content/Videogame_clean_test.csv')
my_df.head()

,clean_text,user_suggestion
0,spookys jump scare mansion scared hear creepy ...,1
1,spookys jump scare mansion good game well sam ...,1
2,spookys jump scare mansion littly iffy control...,1
3,spookys jump scare mansion great game fun colo...,1
4,spookys jump scare mansion not many game cute ...,1


In [0]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17494 entries, 0 to 17493
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   clean_text       17494 non-null  object
 1   user_suggestion  17494 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 273.5+ KB


In [0]:
x = my_df.clean_text
y = my_df.user_suggestion

In [0]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.02, random_state=SEED)

In [0]:
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))

Train set has total 17144 entries with 43.02% negative, 56.98% positive
Validation set has total 350 entries with 43.14% negative, 56.86% positive


In [0]:
from textblob import TextBlob
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time

In [0]:

def accuracy_summary(pipeline, x_train, y_train, x_test, y_test):
    if len(x_test[y_test == 0]) / (len(x_test)*1.) > 0.5:
        null_accuracy = len(x_test[y_test == 0]) / (len(x_test)*1.)
    else:
        null_accuracy = 1. - (len(x_test[y_test == 0]) / (len(x_test)*1.))
    t0 = time()
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    train_test_time = time() - t0
    accuracy = accuracy_score(y_test, y_pred)
    print( "null accuracy: {0:.2f}%".format(null_accuracy*100))
    print( "accuracy score: {0:.2f}%".format(accuracy*100))
    if accuracy > null_accuracy:
        print("model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100))
    elif accuracy == null_accuracy:
        print( "model has the same accuracy with the null accuracy")
    else:
        print( "model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100))
    print( "train and test time: {0:.2f}s".format(train_test_time))
    print( "-"*80)
    return accuracy, train_test_time

In [0]:
cvec = CountVectorizer()
lr = LogisticRegression()
n_features = np.arange(10000,100001,10000)

def nfeature_accuracy_checker(vectorizer=cvec, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=lr):
    result = []
    print (classifier)
    print( "\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print( "Validation result for {} features".format(n))
        nfeature_accuracy,tt_time = accuracy_summary(checker_pipeline, x_train, y_train, x_validation, y_validation)
        result.append((n,nfeature_accuracy,tt_time))
    return result

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer()

In [0]:
%%time
print( "RESULT FOR TRIGRAM WITH STOP WORDS (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tvec,ngram_range=(1, 3))

RESULT FOR TRIGRAM WITH STOP WORDS (Tfidf)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


Validation result for 10000 features
null accuracy: 56.86%
accuracy score: 82.00%
model is 25.14% more accurate than null accuracy
train and test time: 22.30s
--------------------------------------------------------------------------------
Validation result for 20000 features
null accuracy: 56.86%
accuracy score: 82.57%
model is 25.71% more accurate than null accuracy
train and test time: 22.43s
--------------------------------------------------------------------------------
Validation result for 30000 features
null accuracy: 56.86%
accuracy score: 83.43%
model is 26.57% more accurate than null accuracy
train and test ti

In [0]:
test_pred = test['clean_text']

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
x_train_tfidf = tvec.fit_transform(x)
test_validation_tfidf = tvec.transform(test_pred)

In [0]:
x_train_tfidf

<17494x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 10967353 stored elements in Compressed Sparse Row format>

In [0]:
test_validation_tfidf

<8045x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 2154306 stored elements in Compressed Sparse Row format>

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x_vectorizer = vectorizer.fit_transform(x)
test_vectorizer = vectorizer.transform(test_pred)

In [0]:
x_vectorizer

<17494x46631 sparse matrix of type '<class 'numpy.int64'>'
	with 3332293 stored elements in Compressed Sparse Row format>

In [0]:
test_vectorizer

<8045x46631 sparse matrix of type '<class 'numpy.int64'>'
	with 1392560 stored elements in Compressed Sparse Row format>

In [0]:
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
maxscaler = MaxAbsScaler().fit(x_train_tfidf)
X_train_MaxAbsScaler = maxscaler.transform(x_train_tfidf)
test__MaxAbsScaler = maxscaler.transform(test_validation_tfidf)

In [0]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(100)
train_tsvd = tsvd.fit_transform(X_train_MaxAbsScaler)
test_tsvd = tsvd.fit_transform(test__MaxAbsScaler)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_train_tfidf,y, test_size = 0.3, shuffle = True)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_vectorizer,y, test_size = 0.3, shuffle = True)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_train_MaxAbsScaler,y, test_size = 0.3, shuffle = True)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(train_tsvd,y, test_size = 0.3, shuffle = True)

In [0]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [0]:
lr=LogisticRegression(penalty='l2',max_iter=1000,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(xtrain,ytrain)
print(lr_bow)
lr_bow_predict=lr.predict(xtest)
lr_bow_report=classification_report(ytest,lr_bow_predict)
print(lr_bow_report)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
              precision    recall  f1-score   support

           0       0.86      0.83      0.84      2236
           1       0.88      0.90      0.89      3013

    accuracy                           0.87      5249
   macro avg       0.87      0.86      0.87      5249
weighted avg       0.87      0.87      0.87      5249



In [0]:
test_predict_lr = lr_bow.predict(test__MaxAbsScaler)

In [0]:
test_predict_lr_vectorizer = lr_bow.predict(test_vectorizer)

In [0]:
test_predict_lr

array([1, 0, 0, ..., 1, 1, 1])

In [0]:
test_predict_lr_vectorizer

array([0, 0, 0, ..., 1, 1, 1])

In [0]:
pd.DataFrame(test_predict_lr).to_csv('test_all_predict_lr_maxabsscaling.csv')

In [0]:
pd.DataFrame(test_predict_lr_vectorizer).to_csv('test_predict_lr_vectorizer.csv')

In [0]:
from sklearn.feature_selection import chi2
chi2score = chi2(x_train_tfidf, y_train)[0]

In [0]:
  from sklearn.ensemble import RandomForestClassifier
  random_clf = RandomForestClassifier(max_depth=15, random_state=0)
  random_clf.fit(xtrain,ytrain)
  print("\n**Random forest**\n")
  
  print('train accuracy',random_clf.score(xtrain,ytrain))
  print('test accuracy',random_clf.score(xtest,ytest))
  
  y_pred = random_clf.predict(xtest)
  
  from sklearn.metrics import classification_report, confusion_matrix
  print("classification report :  \n ", classification_report(ytest, y_pred))


**Random forest**

train accuracy 0.74822376480196
test accuracy 0.7247094684701848
classification report :  
                precision    recall  f1-score   support

           0       0.82      0.45      0.58      2236
           1       0.70      0.93      0.79      3013

    accuracy                           0.72      5249
   macro avg       0.76      0.69      0.69      5249
weighted avg       0.75      0.72      0.70      5249



In [0]:
from sklearn.tree import DecisionTreeClassifier
dec_clf = DecisionTreeClassifier(random_state=0)
dec_clf.fit(xtrain,ytrain)
print("\n**Random forest**\n")
  
print('train accuracy',dec_clf.score(xtrain,ytrain))
print('test accuracy',dec_clf.score(xtest,ytest))
  
y_pred = dec_clf.predict(xtest)
  
from sklearn.metrics import classification_report, confusion_matrix
print("classification report :  \n ", classification_report(ytest, y_pred))


**Random forest**

train accuracy 0.9999183340138832
test accuracy 0.7384263669270337
classification report :  
                precision    recall  f1-score   support

           0       0.70      0.68      0.69      2236
           1       0.77      0.78      0.77      3013

    accuracy                           0.74      5249
   macro avg       0.73      0.73      0.73      5249
weighted avg       0.74      0.74      0.74      5249



In [0]:
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(xtrain,ytrain)
print(mnb_bow)
mnb_bow_predict=mnb.predict(xtest)
mnb_bow_report=classification_report(ytest,mnb_bow_predict)
print(mnb_bow_report)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
              precision    recall  f1-score   support

           0       0.65      0.74      0.69      2236
           1       0.78      0.71      0.75      3013

    accuracy                           0.72      5249
   macro avg       0.72      0.72      0.72      5249
weighted avg       0.73      0.72      0.72      5249



In [0]:
pred_test_NB_tfidf = mnb_bow.predict(test_validation_tfidf)

In [0]:
pd.DataFrame(pred_test_NB_tfidf).to_csv('pred_test_NB_tfidf.csv')

In [0]:
from sklearn.svm import SVC
SVC_clf = SVC()
SVC_clf.fit(xtrain, ytrain)
SVC_clf_pred = SVC_clf.predict(xtest)
print(classification_report(ytest,SVC_clf_pred))

              precision    recall  f1-score   support

           0       0.73      0.59      0.65      2312
           1       0.72      0.83      0.77      2937

    accuracy                           0.72      5249
   macro avg       0.72      0.71      0.71      5249
weighted avg       0.72      0.72      0.72      5249



In [0]:
from sklearn.svm import LinearSVC
LinearSVC_clf = LinearSVC(random_state=0, tol=1e-5)
LinearSVC_clf.fit(xtrain, ytrain)
LinearSVC_clf_pred = LinearSVC_clf.predict(xtest)
print(classification_report(ytest,SVC_clf_pred))

              precision    recall  f1-score   support

           0       0.73      0.59      0.65      2312
           1       0.72      0.83      0.77      2937

    accuracy                           0.72      5249
   macro avg       0.72      0.71      0.71      5249
weighted avg       0.72      0.72      0.72      5249



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [0]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(xtrain,ytrain)

y_pred = xgb_model.predict(xtest)

print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.70      0.76      2280
           1       0.79      0.89      0.84      2969

    accuracy                           0.81      5249
   macro avg       0.81      0.79      0.80      5249
weighted avg       0.81      0.81      0.80      5249



In [0]:
pred_test_xgb_tfidf = xgb_model.predict(test_validation_tfidf)
pd.DataFrame(pred_test_xgb_tfidf).to_csv('pred_test_xgb_tfidf.csv')

In [0]:
import lightgbm as lgbm
import re

In [0]:
params = {
    'objective' :'binary',
    'learning_rate' : 0.02,
    'num_leaves' : 76,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'binary_logloss'
}

In [0]:
    d_train = lgbm.Dataset(xtrain,ytrain)
    d_valid = lgbm.Dataset(xtest, ytest)
    
    # training with early stop
    bst = lgbm.train(params, d_train, 5000, valid_sets=[d_valid], verbose_eval=50, early_stopping_rounds=100)
    
    # making prediciton for one column
    
    y_pred = np.round(bst.predict(xtest))
    print(classification_report(ytest, y_pred))

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.497919
[100]	valid_0's binary_logloss: 0.43145
[150]	valid_0's binary_logloss: 0.39568
[200]	valid_0's binary_logloss: 0.372823
[250]	valid_0's binary_logloss: 0.357476
[300]	valid_0's binary_logloss: 0.346799
[350]	valid_0's binary_logloss: 0.338685
[400]	valid_0's binary_logloss: 0.333571
[450]	valid_0's binary_logloss: 0.32931
[500]	valid_0's binary_logloss: 0.326763
[550]	valid_0's binary_logloss: 0.325306
[600]	valid_0's binary_logloss: 0.324652
[650]	valid_0's binary_logloss: 0.324005
[700]	valid_0's binary_logloss: 0.323949
[750]	valid_0's binary_logloss: 0.325037
Early stopping, best iteration is:
[688]	valid_0's binary_logloss: 0.323468
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      2274
           1       0.86      0.90      0.88      2975

    accuracy                           0.86      5249
   macro avg       0.86      0.8

In [0]:
y_pred_rounf = np.round(y_pred)

In [0]:
y_pred_rounf

array([1., 1., 0., ..., 0., 1., 1.])

In [0]:
y_pred = bst.predict(xtest)
print(classification_report(ytest, y_pred_rounf))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80      2235
           1       0.84      0.87      0.86      3014

    accuracy                           0.83      5249
   macro avg       0.83      0.83      0.83      5249
weighted avg       0.83      0.83      0.83      5249



In [0]:
test_predict_lgbm = np.round(bst.predict(test_validation_tfidf))
pd.DataFrame(test_predict_lgbm).to_csv('test_predict_lgbm_tfid.csv')